In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import replicaEVSE.datautils as simdu
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data/'
mode = 'PRIVATE_AUTO'
test = False

In [2]:


#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv(datadir+'EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))
    df = df.head(10000)
    df = df.loc[df['mode'] == mode]
    simulation_id = 'dev'

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]
    # take out the mobile and commercial MHDV

### TODO: revisit taking out mobile home owners
df = df[(df['building_type'] != 'mobile') & (df['building_type'] != None)]



###  sample by county: for now use a percent until we have the stock rollover model

`ev_df` should be given to us to run. 

In [3]:
stock_rollover = pd.read_csv(datadir+'ldv_population_output_adjusted.csv')
efficiency = pd.read_csv(datadir+'vehicle_inputs.csv')

In [4]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']
for cartype in personal + commercial:
    for powertrain in  ['EV', 'PHEV']:
        cond = (efficiency['Vehicle_type']==cartype) & ~efficiency['Powertrain'].isin(['ICE-G', 'ICE-D', 'FCEV']) & (efficiency['Powertrain']==powertrain) & (efficiency['Vintage'] == 2022)
        ef = efficiency[cond]['Efficiency'].values[0]
        print(cartype, powertrain, ef)

Personal Sedan EV 0.25
Personal Sedan PHEV 80.0
Personal Crossover EV 0.3
Personal Crossover PHEV 80.0
Personal Truck/SUV EV 0.49
Personal Truck/SUV PHEV 80.0
Commercial Sedan EV 0.25
Commercial Sedan PHEV 80.0
Commercial Crossover EV 0.3
Commercial Crossover PHEV 80.0
Commercial Truck/SUV EV 0.49
Commercial Truck/SUV PHEV 80.0


# extract the population ldv data from the stock rollover model 
 - What to do about housing == 'other'? 

### include both EV and PHEV

In [5]:
ev_cond = stock_rollover['Powertrain'].isin(['EV', 'PHEV']).reset_index(drop=True)
# ev_cond = (stock_rollover['Powertrain']=='EV') | (stock_rollover['Powertrain']=='PHEV')
# nev_df = stock_rollover[(stock_rollover['Powertrain']=='EV')].copy()
nev_df = stock_rollover[ev_cond].copy()
nev_df = nev_df[nev_df['domicile'] != 'other'].copy()
nev_df.drop(columns=['Unnamed: 0'], inplace=True)
print(len(nev_df))


924


See the change in stock over the years

In [6]:
nev_df[['2022', '2035']]

,2022,2035
0,0,1
3,0,0
4,0,5
7,0,1
8,0,0
...,...,...
1839,98,14859
1840,34,22314
1843,18,5020
1844,133,83107


We ingest the number of vehicles in each segment for each year and sample from that subset of the trips table for each county segment and year. This takes > 300 minutes for all the years and segments

# HERE: main debugging for the sampling

- previously each year was the same size

In [7]:
nev_df

,County,Vehicle_type,domicile,Powertrain,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
0,Adams,Commercial Crossover,mfh,EV,0,0,0,0,0,0,0,1,1,1,1,1,1,1
3,Adams,Commercial Crossover,mfh,PHEV,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Adams,Commercial Crossover,sfh,EV,0,0,1,1,1,2,2,2,3,3,3,5,5,5
7,Adams,Commercial Crossover,sfh,PHEV,0,1,1,1,1,1,1,1,1,1,1,1,1,1
8,Adams,Commercial Sedan,mfh,EV,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1839,Yakima,Personal Sedan,sfh,PHEV,98,2433,4815,6453,7994,9117,10046,10703,11606,12252,12842,13552,14206,14859
1840,Yakima,Personal Truck/SUV,mfh,EV,34,1780,3504,5143,6768,8393,9963,11285,12851,14524,16279,18064,19987,22314
1843,Yakima,Personal Truck/SUV,mfh,PHEV,18,1063,1757,2265,2617,2945,3170,3367,3593,3845,4102,4380,4667,5020
1844,Yakima,Personal Truck/SUV,sfh,EV,133,6521,12813,18796,24713,30637,36335,41141,46932,53211,59811,66703,74132,83107


In [8]:
def segment_efficiency(segment):
    """This simply returns the expected efficiency (kwh/mi) for a given segment.
    It is quite simple now but could be expanded to include more complex
    efficiency calculations over the years.

    Args:
        segment (str): The segment of the vehicle

    Returns:
        float: efficiency in kWh/mile
    """

    # get the efficiency for the segment
    if "Sedan" in segment:
        eff = 0.25
    elif "Crossover" in segment:
        eff = 0.30
    elif 'Truck' in segment:
        eff = 0.49
    else:
        print('no match')
    return eff

In [9]:
# get the unique people in the dataframe
pop_df = df.drop_duplicates(subset=['person_id'])[['person_id', 'destination_county', 'building_type']]

In [10]:
county = 'King'

In [11]:
final_df = simdu.sample_people_by_county(df, nev_df, year=2025, fraction=None)

0 Adams mfh Commercial Crossover EV
0 Adams mfh Commercial Crossover PHEV
1 Adams sfh Commercial Crossover EV
1 Adams sfh Commercial Crossover PHEV
0 Adams mfh Commercial Sedan EV
0 Adams mfh Commercial Sedan PHEV
6 Adams sfh Commercial Sedan EV
3 Adams sfh Commercial Sedan PHEV
12 Adams mfh Commercial Truck/SUV EV
7 Adams mfh Commercial Truck/SUV PHEV
60 Adams sfh Commercial Truck/SUV EV
35 Adams sfh Commercial Truck/SUV PHEV
5 Adams mfh Personal Crossover EV
3 Adams mfh Personal Crossover PHEV
28 Adams sfh Personal Crossover EV
19 Adams sfh Personal Crossover PHEV
239 Adams mfh Personal Sedan EV
144 Adams mfh Personal Sedan PHEV
984 Adams sfh Personal Sedan EV
590 Adams sfh Personal Sedan PHEV
421 Adams mfh Personal Truck/SUV EV
256 Adams mfh Personal Truck/SUV PHEV
1731 Adams sfh Personal Truck/SUV EV
1041 Adams sfh Personal Truck/SUV PHEV
0 Asotin mfh Commercial Crossover EV
0 Asotin mfh Commercial Crossover PHEV
0 Asotin sfh Commercial Crossover EV
0 Asotin sfh Commercial Crossove

In [ ]:

for year in [2023]:
    segment_persons = []

    # we loop over segments in order to assign one car type per person.
    for segment in ['Commercial Crossover', 'Commercial Truck/SUV']:
        
        # convert to readable string for simulation id
        seg_string = segment.replace(" ", "_").lower().replace("/suv", "")
        simulation_id = f'{seg_string}_{str(year)}'
        print('+------------------------------------+')
        print(f"Current segment and year: {simulation_id}")
        print('+------------------------------------+')

        # get the efficiency for the segment
        eff = segment_efficiency(segment)

        # this is defined above and is a cut from the stock rollover raw data
        # it is the number of vehicles of a given seg per year. 
        segment_condition = (nev_df['Vehicle_type']==segment) & (nev_df[str(year)]==str(year))
        nev_df_sub = nev_df[segment_condition].copy()
        print(len(nev_df_sub))
        
        # we dont want to include the same people in multiple segments.
        # this results in not enough people in some small counties...
        #if len(segment_persons) == 0:
        #    df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year) 
        #else:
        #    df = df[~df['person_id'].isin(segment_persons)]    
        # ~3 mins
        df_county_subset = simdu.sample_people_by_county(df, nev_df_sub, year=year)
        df_county_subset['efficiency'] = eff
        df_county_subset['segment'] = seg_string
        # segment_persons.extend(df_county_subset['person_id'].tolist())
        # ddf = dd.from_pandas(df_county_subset, npartitions=500)
        df_county_subset.to_parquet(os.path.join(datadir, f'county_samples/county_sample_{simulation_id}.parquet'), 
                                    engine='pyarrow',
                                    compression='snappy')
        segment_persons.append(df_county_subset['person_id'].tolist())

In [ ]:
year

In [ ]:
cnty_df['destination_county'].value_counts()

In [ ]:
24725/455888 

In [ ]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']

for year in np.arange(2022, 2036, 1):
    segment_persons = []
    for segment in personal+commercial:
        seg_string = segment.replace(" ", "_").lower().replace("/suv", "")
        simulation_id = f'{seg_string}_{str(year)}'
        print(simulation_id)
        if "Sedan" in segment:
            eff = 0.25
        elif "Crossover" in segment:
            eff = 0.30
        elif 'Truck' in segment:
            eff = 0.49
        else:
            print('no match')
            eff = 0.3
        print(eff)
        nev_df = stock_rollover[stock_rollover['Powertrain']=='EV'].copy()
        nev_df_seg = nev_df[nev_df['Vehicle_type']==segment].copy()
        
        
        # we dont want to include the same people in multiple segments.
        # this results in not enough people in some small counties...
        #if len(segment_persons) == 0:
        #    df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year) 
        #else:
        #    df = df[~df['person_id'].isin(segment_persons)]    
        # ~3 mins
        df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year)
        df_county_subset['efficiency'] = eff
        segment_persons.extend(df_county_subset['person_id'].tolist())
        # ddf = dd.from_pandas(df_county_subset, npartitions=500)
        df_county_subset.to_parquet(os.path.join(datadir, f'county_sample_{simulation_id}.parquet'), 
                                    engine='pyarrow',
                                    compression='snappy')